First need to convert the dataset to jsonl format that is supported by OpenAI.
System prompts for training, user role for hate speech content and assistant rolw for Counter narratives.

In [ ]:
import pandas as pd
import json

df = pd.read_excel("dataset.xlsx")

data = []

# System prompt in English
system_prompt_en = """
You are a knowledgeable and respectful assistant trained to generate professional and constructive counter narratives in Tamil to counter online hate speech.
Your responses must be empathetic, socially inclusive, and free from hate, bias, or sarcasm.
Your responses must promote dignity, empathy, and social awareness without using moral superiority or counter-attacks.
Use formal Tamil language with tone and style used in social media platforms like Twitter, Facebook, Youtube.
"""

# System prompt in tamil language (Used for training the model)
system_prompt = """
ஆன்லைனில் வெறுப்புப் பேச்சுகளை எதிர்கொள்வதற்காக தமிழில் தொழில்முறை மற்றும் ஆக்கபூர்வமான எதிர் விவரிப்புகளை உருவாக்க பயிற்சி பெற்ற அறிவுள்ள மற்றும் மரியாதைக்குரிய உதவியாளர் நீங்கள்.
உங்கள் பதில்கள் பச்சாதாபமானதாகவும், சமூகத்தை உள்ளடக்கியதாகவும், வெறுப்பு, சார்பு அல்லது கிண்டல் இல்லாததாகவும் இருக்க வேண்டும்.
உங்கள் பதில்கள் தார்மீக மேன்மை அல்லது எதிர் தாக்குதல்களைப் பயன்படுத்தாமல் கண்ணியம், பச்சாதாபம் மற்றும் சமூக விழிப்புணர்வை ஊக்குவிக்க வேண்டும்.
ட்விட்டர், பேஸ்புக், யூடியூப் போன்ற சமூக ஊடக தளங்களில் பயன்படுத்தப்படும் தொனி மற்றும் பாணியுடன் முறையான தமிழ் மொழியைப் பயன்படுத்தவும்.
"""

for _, row in df.iterrows():
    if pd.notnull(row['Hate speech']) and pd.notnull(row['Counter Narrative from dataset']):
        data.append({
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": row['Hate speech']},
                {"role": "assistant", "content": row['Counter Narrative from dataset']}
            ]
        })

# OpenAI requires file to be saved to jsonl format
with open("dataset.jsonl", "w", encoding='utf-8') as f:
    for item in data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

Next step, upload the above generated file to OpenAI inorder to fine-tune the model.

In [ ]:
import openai
from openai import OpenAI
from google.colab import userdata
import os

api_key = userdata.get('OPENAI_API_KEY')


client = openai.OpenAI(api_key=api_key)



Upload the dataset file to Open API for fine-tuning the model.

In [ ]:
with open("dataset.jsonl", "rb") as f:
    file = client.files.create(file=f, purpose="fine-tune")

print("Uploaded file ID:", file.id)

FIne tune the model with the uploaded data

In [ ]:
print(f"Fine tuning the model using file: {file.id}")

job = client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)

print("Fine-tuning Job ID:", job.id)

In [ ]:
job = client.fine_tuning.jobs.retrieve(job.id)
print("Status:", job.status)
print("Fine-tuned model ID:", job.fine_tuned_model)
model = job.fine_tuned_model

Generate HS-CN pairs using fine-tuned model

In [ ]:
import openai
import pandas as pd
import time
import json

model = "<FINE-TUNED-MODEL-ID>"


system_prompt_en = """
You are a Tamil language expert assistant trained to generate realistic social media hate speech and appropriate counter narratives.
You will be given a category: either "Misogyny" or "Xenophobic".

Your task:

1. Generate a single realistic Tamil hate speech statement in the given category that:
  Uses raw, aggressive, unfiltered informal Tamil as found in real social media (e.g., YouTube, Facebook, TikTok, Twitter)
  May include slurs, personal attacks, or sexist/casteist/communal insults as seen in public content.
  Uses raw, aggressive, unfiltered informal Tamil as found in real social media (e.g., YouTube, Facebook, TikTok, Twitter)
  Feels natural to how Tamil speakers express hate — including slang, taunts, emotion, and informal style.

2. Generate a counter narrative in Tamil that:
  Responds calmly and respectfully while rejecting the hate.
  Uses reasoning, empathy, or assertive but civil tone to challenge the prejudice.
  Avoids mocking, sarcasm, or violent language — must remain constructive.

Respond strictly in the following JSON format: {
  "category": "<Misogyny or Xenophobic>",
  "hate_speech": "<Tamil hate speech statement>",
  "counter_narrative": "<Tamil counter narrative>"
}

Only respond in Tamil language. Do not include explanations, translations, or any extra text.
"""

system_prompt = """
நீங்கள் யதார்த்தமான சமூக ஊடக வெறுப்புப் பேச்சு மற்றும் பொருத்தமான எதிர் கதைகளை உருவாக்க பயிற்சி பெற்ற தமிழ் மொழி நிபுணர் உதவியாளர்.

உங்களுக்கு ஒரு வகை வழங்கப்படும்: "பெண் வெறுப்பு" அல்லது "அயல்நாட்டு வெறுப்பு".

உங்கள் பணி:

1. கொடுக்கப்பட்ட பிரிவில் ஒரு யதார்த்தமான தமிழ் வெறுப்புப் பேச்சு அறிக்கையை உருவாக்குதல். இந்த அறிக்கை:
  - சமூக ஊடக தளங்களில் காணப்படும் பச்சையான, ஆக்ரோஷமான தமிழைப் பயன்படுத்த வேண்டும்..
  - சாதி, பாலியல், பிராந்திய அவமதிப்புகள், தனிப்பட்ட தாக்குதல்கள் ஆகியவை சேர்க்கலாம்.
  - பேச்சு இயல்பான, உணர்ச்சிவயப்பட்டதாக இருக்க வேண்டும்.

2. அந்த வெறுப்புப் பேச்சுக்கு நேரடியாக பதிலளிக்கும் எதிர்க் கதையை உருவாக்கவும்
  - அமைதியாகவும் மரியாதையுடனும் இருக்க வேண்டும்.
  - பாரபட்சம் அல்லது தவறான தகவலை சவால் செய்ய பகுத்தறிவு, பச்சாதாபம் அல்லது வலுவான ஆனால் மரியாதைக்குரிய மொழியைப் பயன்படுத்தவும்.
  - தவறான தகவலை எதிர்க்க பகுத்தறிவு அல்லது பச்சாதாபத்தைப் பயன்படுத்தவும்.
  - வன்முறையற்ற, அவமதிப்பற்ற, இயல்பான ஒலியுடன் இருக்க வேண்டும்.

பின்வரும் வடிவத்தில் கண்டிப்பாக பதிலளிக்கவும்:
{
"வகை": "<தவறான வெறுப்பு அல்லது இனவெறி>",
"வெறுப்பு_பேச்சு": "<தமிழ் வெறுப்புப் பேச்சு அறிக்கை>",
"எதிர்_கதை": "<தமிழ் எதிர் கதை"
}

தமிழ் மொழியில் மட்டும் பதிலளிக்கவும். விளக்கங்கள், மொழிபெயர்ப்புகள் அல்லது கூடுதல் உரையைச் சேர்க்க வேண்டாம்.

"""

data = []

# Only considering Misogyny or Xenophobic contents
categories = ["Misogyny", "Xenophobic"]

# Vi dataset
pairs_per_category = 100  # Generate the dataset as required based on the iteration

# Generate data
for category in categories:
    for _ in range(pairs_per_category):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f'"{category}" வகைப்படியில் ஒரு ஜோடியை மட்டுமே தமிழில் உருவாக்குங்கள்.'}
                ],
                temperature=0.8
            )
            output_text = response.choices[0].message.content.strip()

            try:
              # Try to parse the JSON response safely
              parsed = json.loads(output_text)

              # Access fields and store them in your data list
              data.append({
                  "Category": parsed.get("category", ""),
                  "Hate speech": parsed.get("hate_speech", ""),
                  "Counter narrative": parsed.get("counter_narrative", "")
              })

            except json.JSONDecodeError as parse_error:
              print(f"Parsing error: {parse_error}\nRaw response: {output_text}")
              time.sleep(1)

            # output text will be in json format
            # try:
            #     parsed = eval(output_text)
            #     data.append({
            #         "Category": parsed["category"],
            #         "Hate speech": parsed["hate_speech"],
            #         "Counter narrative": parsed["counter_narrative"]
            #     })
            # except Exception as parse_error:
            #     print(f"Parsing error: {parse_error} \nRaw response: {output_text}")

            # time.sleep(1)  # Pause to avoid rate limits
        except Exception as e:
            print(f"API error: {e}")
            time.sleep(5)

# Save to Excel
df = pd.DataFrame(data)
df.to_excel("dataset.xlsx", index=False)